In [22]:
import sklearn as sk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [23]:
train_raw = pd.read_csv('/home/david/kaggle/kaggle-titanic/train.csv')
train = train_raw.copy()
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
#Data Preprocessing
def cleanData(df):

    df.columns = df.columns.str.strip() #Strip any stray spaces in the column titles

    df = df.drop(['Ticket', 'Name', 'Cabin'], axis=1) #Drop columns that aren't expected to have any predictive power
    df['Age'] = df['Age'].fillna(df['Age'].median()) #Fill Nan values with Median

    #train = train.fillna(0, axis=0) #fill all rows that include null values
    df = df.set_index(df.PassengerId).drop('PassengerId', axis=1)

    
    #Convert categorical features to dummy features
    df['Sex'] = pd.get_dummies(df['Sex'])
    df = pd.concat([df, pd.get_dummies(df['Embarked'])], axis=1) 
    df = df.drop('Embarked', axis=1)
    
    return df.astype(float)

In [25]:
train = cleanData(train)
train = train.dropna(axis=0)
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
PassengerId,,,,,,,,,,
1,0.0,3.0,0.0,22.0,1.0,0.0,7.2500,0.0,0.0,1.0
2,1.0,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,0.0
3,1.0,3.0,1.0,26.0,0.0,0.0,7.9250,0.0,0.0,1.0
4,1.0,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,1.0
5,0.0,3.0,0.0,35.0,0.0,0.0,8.0500,0.0,0.0,1.0


In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 10 columns):
Survived    891 non-null float64
Pclass      891 non-null float64
Sex         891 non-null float64
Age         891 non-null float64
SibSp       891 non-null float64
Parch       891 non-null float64
Fare        891 non-null float64
C           891 non-null float64
Q           891 non-null float64
S           891 non-null float64
dtypes: float64(10)
memory usage: 76.6 KB


In [27]:
X_train = train.drop('Survived', axis=1)
y_train = train['Survived']

#X_fit, X_val, y_fit, y_val = train_test_split(X_train, y_train, test_size=0.9, random_state=42)

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

SVCpipe = Pipeline([
    ('scaler', StandardScaler()),
    ('linearSVC', LinearSVC(C=1, loss="hinge"))
])

SVCpipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linearSVC', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

In [29]:
from sklearn.metrics import accuracy_score

print("Training Accuracy: ")
print(accuracy_score(y_train, SVCpipe.predict(X_train)))

Training Accuracy: 
0.786756453423


In [31]:
X_test = pd.read_csv('/home/david/kaggle/kaggle-titanic/test.csv')
X_test_clean = cleanData(X_test)
X_test_clean = X_test_clean.fillna(0, axis=0) #fill all rows that include null values

predictions = SVCpipe.predict(X_test_clean)

outputDF = pd.DataFrame(data={'Survived':predictions}, index=X_test_clean.index)
outputDF = outputDF.astype(int)

outputDF.to_csv('/home/david/kaggle/kaggle-titanic/submission.csv')

### Trying to do other things:

In [36]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(SVCpipe, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

print("Scores:", rmse_scores)
print("Mean:", rmse_scores.mean())
print("Standard Deviation:", rmse_scores.std())

Scores: [ 0.43461349  0.45946829  0.47404546  0.39661489  0.44971901  0.47404546
  0.48575205  0.50835712  0.43704832  0.48850421]
Mean: 0.460816832446
Standard Deviation: 0.0308169684602


In [37]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [1, 2, 4, 6, 8, 9]},
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [1, 2,3,4]},
   ]

forest_clf = RandomForestClassifier()

rand_search = RandomizedSearchCV(forest_clf, param_grid[0], cv=5, scoring='neg_mean_squared_error')

rand_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': [3, 10, 30], 'max_features': [1, 2, 4, 6, 8, 9]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='neg_mean_squared_error',
          verbose=0)

In [34]:
rand_search.cv_results_

{'mean_fit_time': array([ 0.04432187,  0.0053503 ,  0.01226263,  0.00514078,  0.03592305,
         0.01576514,  0.00452013,  0.00489163,  0.00443573,  0.013588  ]),
 'mean_score_time': array([ 0.00266676,  0.00056505,  0.00114641,  0.00056911,  0.00266638,
         0.001122  ,  0.00061479,  0.0006073 ,  0.00060105,  0.00114636]),
 'mean_test_score': array([-0.19753086, -0.1952862 , -0.21099888, -0.21885522, -0.18181818,
        -0.18518519, -0.23681257, -0.22783389, -0.22783389, -0.19304153]),
 'mean_train_score': array([-0.01963812, -0.05976453, -0.03423148, -0.05330622, -0.01907672,
        -0.03619383, -0.05667661, -0.05443375, -0.05723487, -0.03226361]),
 'param_max_features': masked_array(data = [8 9 1 8 2 9 1 4 2 4],
              mask = [False False False False False False False False False False],
        fill_value = ?),
 'param_n_estimators': masked_array(data = [30 3 10 3 30 10 3 3 3 10],
              mask = [False False False False False False False False False False],
   

In [35]:
rand_search.best_estimator_.feature_importances_

array([ 0.08279464,  0.27754874,  0.25957838,  0.04880187,  0.03835303,
        0.25855228,  0.01136197,  0.00746286,  0.01554621])